\# Lab 5. Transformers

---

How to Use This Notebook
---

**Recommended Setup**
- For the best experience, **run this notebook on [Google Colab](https://colab.research.google.com/)**—especially if your local machine is slow.  
- In Colab, **enable GPU support** by going to:  
  `Runtime > Change runtime type > Hardware accelerator > GPU`


**Homework Tasks**

 - Homework tasks are clearly marked throughout the notebook in the following format:

   > ---

   > <span style="color:red"><b>TASK X</b> - [<i>some text</i>]:</span>

   > ---

   > ```Your code ....```

   > ---

   > *End of Task X.* [*Instructions for passing*]

 - For each task:
   - **Complete the code** where indicated.
   - **Upload the required results** from each task to **Homework 5 – Code** on [NextIlearn](https://nextilearn.dsv.su.se).

 - Once you've finished all the tasks:
   Submit your **entire completed notebook (including your code!)** to **Homework 5 – Notebook** on [NextIlearn](https://nextilearn.dsv.su.se).

**Important:**  
Your submission will **only be graded if both files** (code + notebook) are uploaded **before the deadline**. Late submissions are **not accepted**, regardless of technical issues like bad internet connection.

---

This lab introduces students to working with pre-trained BERT embeddings, fine-tuning BERT for classification, and extending BERT with a simple autoregressive head.

\#\# Objectives
- Extract contextual embeddings using a pre-trained BERT model.
- Fine-tune BERT on a downstream classification task.
- Build a simple autoregressive extension of BERT.


In [ ]:
%%capture
# Install necessary libraries
!pip install transformers torch scikit-learn datasets matplotlib

# Unzip data
#!unzip data.zip

In [ ]:
# Import required libraries
import torch
import numpy as np
import pandas as pd
from tqdm import tqdm
from datasets import load_dataset
from torch.utils.data import DataLoader
import torch.nn.functional as F

\#\# 1. Using Pre-trained BERT Embeddings

Let's load a pre-trained BERT model and extract token-level and sentence-level embeddings. [Huggingface's `transformers` library](https://huggingface.co/transformers]) provides easy access to most open source transformer models. The most convenient way is using their `Auto`-classes, which work with most models:

* `transformers.AutoTokenizer` provides access to the pretrained tokenizer (i.e. including the vocabulary used for training the model)
* `transformers.AutoModel` provides access to the pretrained model in its base configuration (i.e. including the trained parameters)

Both provide the `.from\_pretrained()` method, which automatically instantiates the correct class and downloads the pretrained data based on a model identifier:

*   `'bert-base-uncased'`: normal sized BERT ([Devlin et al, 2019](https://doi.org/10.48550/arXiv.1810.04805))
*   `'bert-small-uncased'`: a smaller version of BERT ([Devlin et al, 2019](https://doi.org/10.48550/arXiv.1810.04805))
*   `'FacebookAI/xlm-roberta-base'`: a BERT-sized multilingual model ([Conneau et al, 2019](https://doi.org/10.48550/arXiv.1911.02116))
*   `'gpt2'`: a predecessor of today's GPT-4o, the backbone of ChatGPT ([Radford et al, 2019](https://cdn.openai.com/better-language-models/language\_models\_are\_unsupervised\_multitask\_learners.pdf))

An extensive list of models can be found at [https://huggingface.co/models](https://huggingface.co/models).

In [ ]:
from transformers import AutoTokenizer, AutoModel

\#\#\# **1. Step:** Load the pretrained tokenizer

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
type(tokenizer)

Let's encode some sample sentences to see how it works:

In [ ]:
sentences = ["The quick brown fox jumps over the lazy dog.", "A stitch in time saves nine."]

# tokenize the sentences:
inputs = tokenizer(sentences,
  return_tensors='pt',    # return the output of this function as pytorch tensors.
                          # Other options: 'np' -> numpy
                          #                'tf' -> tensorflow

  padding='max_length',   # pad the sentences to context length of the model.
                          # Other options: 'longest' / True     -> pad to longest length in batch
                          #                'do_not_pad' / False -> no padding

  truncation=True         # Options: 'longest_first' / True    -> Truncate to a maximum length specified with the argument max_length or to the maximum acceptable input length for the model if that argument is not provided.
                          #          'do_not_truncate' / False -> No truncation (i.e., can output batch with sequence lengths greater than the model maximum admissible input size)
)

inputs

`inputs` is a dictionary-like object including all the information needed for the transformer:

\#\#\#\# **Member `'input\_ids'`**: the actual tokens!

In [ ]:
# in encoded form:
inputs['input_ids']

In [ ]:
# the shape is number of input texts x sequnece length:
inputs['input_ids'].shape

In [ ]:
# can be converted back to text:
[tokenizer.decode(ids) for ids in inputs['input_ids']]

[huggingface tokenizer objects](https://huggingface.co/docs/transformers/main\_classes/tokenizer) contain information on the model specific special tokens:

In [ ]:
tokenizer.special_tokens_map

In [ ]:
# Unknown token -> Encodes tokens that have not occured in the training data
tokenizer.unk_token, tokenizer.unk_token_id

In [ ]:
# classification token -> Starts a sequence. Because of this, other transformers
# usually use "beginning of sequence" ([bos]) instead. For BERT-like models
# it corresponds to the position of the classification output.
tokenizer.cls_token, tokenizer.cls_token_id

In [ ]:
# Separator token -> Separates two sentences for the next sentence prediction task
# after pretraining usually used to end the input sequence. Because of this,
# other transformers usually use "end of sequence" ([eos]) instead.
tokenizer.sep_token, tokenizer.sep_token_id

In [ ]:
# Padding token -> Pads sequences to the full input length of the transformer.
tokenizer.pad_token, tokenizer.pad_token_id

In [ ]:
# Mask token -> For the masked language modelling pretraining task. Rarelly used
# after pretraining.
tokenizer.mask_token, tokenizer.mask_token_id

\#\#\#\# **Member `'attention\_mask'`**: a mask specifing the position of  non-padded input tokens in `'input\_ids'`!

In [ ]:
inputs['attention_mask']

In [ ]:
inputs['attention_mask'].shape

In [ ]:
[tokenizer.decode(ids[mask == 1]) for ids, mask in zip(inputs['input_ids'], inputs['attention_mask'])]

In [ ]:
[tokenizer.decode(ids[mask == 0]) for ids, mask in zip(inputs['input_ids'], inputs['attention_mask'])]

\#\#\#\# **Member `'token\_type\_ids'`**: a mask specifing the position of the two sentences in `'input\_ids'` for the next sentence prediction pretraining task!

*Usually not needed after pretraining!*


In [ ]:
inputs['token_type_ids']

In [ ]:
inputs['token_type_ids'].shape

\#\#\# **2. Step:** Load the pretrained model

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

In [ ]:
# load model:
model = AutoModel.from_pretrained('bert-base-uncased')
model.to(device)
model.eval()

model

The `config` property contains information about the model instance:

In [ ]:
model.config

We can use this model rightaway, since it is already pretrained:

In [ ]:
with torch.no_grad():
  outputs = model(
      input_ids=inputs['input_ids'].to(device),
      attention_mask=inputs['attention_mask'].to(device),
      output_hidden_states=True,                # return the hidden states after each transformer layer (default: False)
      output_attentions=True                    # return the self-attention weights (default: False)
  )
outputs.keys()

`outputs` is again a dictionary-like object:

\#\#\#\# **Member `'last\_hidden\_state'`**: the output of the last self-attention layer (i.e. before pooling)!

In [ ]:
outputs['last_hidden_state']

In [ ]:
outputs['last_hidden_state'].shape

\#\#\#\# **Member `'pooler\_output'`**: the output of the pooling layer!

The pooler is a single linear layer with a tanh activation:

```Python
class BertPooler(nn.Module):
    def \_\_init\_\_(self, config):
        super().\_\_init\_\_()
        self.dense = nn.Linear(config.hidden\_size, config.hidden\_size)
        self.activation = nn.Tanh()
    
    def forward(self, hidden\_states):
        \# We "pool" the model by simply taking the hidden state corresponding
        \# to the first token.
        first\_token\_tensor = hidden\_states[:, 0]
        pooled\_output = self.dense(first\_token\_tensor)
        pooled\_output = self.activation(pooled\_output)
        return pooled\_output
```

This layer recieves the last hidden state corresponding to the `[CLS]` token as the input. Remember that the next sentence prediction task is trained on this output for BERT.

(see [BERT source code](https://huggingface.co/transformers/v3.0.2/\_modules/transformers/modeling\_bert.html))

In [ ]:
outputs['pooler_output']

In [ ]:
outputs['pooler_output'].shape

\#\#\#\# **Member `'hidden\_states'`**: all the hidden states after each transformer layer!

*This is only returned because we specified* `output\_hidden\_states=True`*!*

In [ ]:
type(outputs['hidden_states']), len(outputs['hidden_states'])

In [ ]:
[t.shape for t in outputs['hidden_states']]

\#\#\#\# **Member `'attentions'`**: all the attention weights of each head in each transformer layer!

*This is only returned because we specified* `output\_attentions=True`*!*

In [ ]:
type(outputs['attentions']), len(outputs['attentions'])

In [ ]:
[t.shape for t in outputs['attentions']]

Let's have a look at the average attention in the last layer:

In [ ]:
import matplotlib.pyplot as plt

for i in range(2):
  fig, axs = plt.subplots(ncols=12, figsize=(20, 5))
  for layer, ax in enumerate(axs):
    # get attention weights of last transformer layer:
    aw = outputs['attentions'][layer][i].cpu()

    # average over heads:
    aw = aw.mean(dim=0)

    # remove padding tokens:
    mask = inputs['attention_mask'][i]
    aw = aw[mask == 1, :][:, mask == 1]

    # create labels:
    labels = tokenizer.convert_ids_to_tokens(inputs['input_ids'][i][mask == 1])
    x = np.arange(len(labels))

    ax.imshow(aw.detach().numpy())
    ax.set_xticks(ticks=x, labels=labels, rotation=90)
    ax.set_yticks(ticks=x, labels=['']*len(x))
    ax.set_title(f'Layer {layer+1}')

  axs[0].set_yticks(ticks=x, labels=labels)
  plt.tight_layout()
  plt.show()

\#\#\# BERT 模型一般会返回什么内容呢？

outputs.last\_hidden\_state       ✅ 每个 token 的向量（最常用）


outputs.pooler\_output           ✅ 整句的向量（做分类时用）


Note how every attention layer focuses on different token combinations.

\#\#\# Example: using pretrained models as embeddings without furter fine-tuning

Let's load a dataset:

From (https://paperswithcode.com/dataset/ag-news)[https://paperswithcode.com/dataset/ag-news]:
> *AG News (AG’s News Corpus) is a subdataset of AG's corpus of news articles constructed by assembling titles and description fields of articles from the 4 largest classes (“World”, “Sports”, “Business”, “Sci/Tech”) of AG’s Corpus. The AG News contains 30,000 training and 1,900 test samples per class.*

In [ ]:
from datasets import load_dataset

labels = ["World", "Sports", "Business", "Sci/Tech"]
data = load_dataset("ag_news", split='train[:2000]')
data

In [ ]:
data[0]

Add a tokenization step to the pipeline:

In [ ]:
def tokenize_function(example):
    #这里的tokenizer会自动给我的结果加上'input_ids', 'token_type_ids', 'attention_mask'
    #其中'input_ids'是我的上下文向量
    #'token_type_ids'是可选的，token数据类型
    #'attention_mask'注意力标记，哪些向量应该被注意
    return tokenizer(example['text'], truncation=True, padding='max_length')

data = data.map(tokenize_function, batched=True)
data

Hide irrelevant features:

In [ ]:
#bert需要'input_ids','attention_mask', 'label'
data.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
data[0]

Calculate BERT embeddings:

In [ ]:
x, y = [], []

# Hugging Face的所有数据集都实现了Dataloader

# predict:
for batch in tqdm(DataLoader(data, batch_size=32)):
  with torch.no_grad():
    y.extend(batch.pop('label').cpu().numpy())
    #items()返回一个可迭代对象包含每一对键和值
    batch = {k: v.to(device) for k, v in batch.items()}
    #model(**batch) ==  model(batch['input_ids'],batch["attention_mask"])
    x.extend(model(**batch).pooler_output.cpu().numpy())

# convert lists to numpy:
x = np.array(x)
y = np.array(y)

x.shape, y.shape

In [ ]:
from sklearn.model_selection import ShuffleSplit
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import f1_score

f1_knn, f1_rnd = [], []
ks = np.arange(1, 21)
for k in tqdm(ks):
  f1_knn.append([])
  f1_rnd.append([])

  # Monte-Carlo cross-validation with 50 splits:
  mc = ShuffleSplit(n_splits=50, test_size=0.25, train_size=None)
  for idx_train, idx_test in mc.split(x):

    # knn classifier:
    knn = KNeighborsClassifier(n_neighbors=k).fit(x[idx_train], y[idx_train])
    y_pred = knn.predict(x[idx_test])
    f1_knn[-1].append(f1_score(y[idx_test], y_pred, average='macro'))

    # random baseline:
    y_rnd = np.random.randint(0, len(labels), size=len(y[idx_test]))
    f1_rnd[-1].append(f1_score(y[idx_test], y_rnd, average='macro'))

# convert to numpy:
f1_knn = np.array(f1_knn)
f1_rnd = np.array(f1_rnd)

# plot:
plt.plot(np.mean(f1_knn, axis=1), label='KNN')
plt.plot(np.mean(f1_rnd, axis=1), label='Random')
plt.legend()
plt.xticks(ticks=ks-1, labels=ks)
plt.xlabel('$k$')
plt.ylabel('F$_1$-score')
plt.show()

In [ ]:
from typing import Iterable
from transformers import BertModel, BertTokenizer
from numpy.typing import NDArray
from sklearn.metrics.pairwise import cosine_similarity

---

<span style="color:red"><b>TASK 1</b> - Text similarity:</span>

---

Write a function to compute text similarity using cosine similarity between BERT embeddings of different texts. The function should take a list of $n$ strings as its input and return a similarity matrix $\in \mathbb\{R\}^\{n \times n\}$.

Afterward, use this function to compute the similarity for the sentences in `data.zip/task1/sentences.csv` and upload your solution to NextIlearn.

**Hint:** See [here](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.cosine\_similarity.html) for documentation of the `sklearn.metrics.pairwise.cosine\_similarity` function.

In [ ]:
def text_similarity(texts:Iterable[str], model:BertModel, tokenizer:BertTokenizer, device:torch.device) -> NDArray[np.float32]:
  # do stuff here
    model.eval()
    inputs = tokenizer(texts, padding= True, truncation= True, return_tensors= "pt")
    inputs = {k : v.to(device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)
        embeddings = outputs.pooler_output

    embeddings = embeddings.cpu().numpy()
    similarity = cosine_similarity(embeddings).astype(np.float32)
    return similarity

In [ ]:
# calculate similarity for sample sentences:
sentences  = pd.read_csv('data/task1/sentences.csv', index_col=0)
similarity = text_similarity(sentences['sentences'].values.tolist(), model, tokenizer, device)
pd.DataFrame(similarity).to_csv('similarity.csv')

---

*End of Task 1. Upload your final predictions (the file* `similarity.csv` *) to* **Homework 2 - Code** *on* **NextIlearn**

When you are done with Task 1, feel free to play around with the function a little:

In [ ]:
plt.imshow(text_similarity(['Paul is cooking dinner for his friend.', 'Maria is cooking dinner for her friend.', 'Stockholm is a beautiful city!'], model, tokenizer, device))

\#\# 2. Fine-tuning BERT for Text Classification

While the KNN classifier based on BERT embeddings is already performing well, we can improve on them by fine-tuning the model for our task. We will reuse the data preprocessing pipeline we used before!

In [ ]:
# split data in training and test set:
split_data = data.train_test_split(test_size=0.25)
split_data

In [ ]:
# create dataloaders:
train_dataloader = DataLoader(split_data['train'], batch_size=16)
test_dataloader = DataLoader(split_data['test'], batch_size=16)

For classification, we need a different "head" on our model. Huggingface provides different setups for each model (see [huggingface documentation](https://huggingface.co/transformers/v3.0.2/model\_doc/auto.html)):

*   `transformers.AutoModel`
*   `transformers.AutoModelForPreTraining`
*   `transformers.AutoModelWithLMHead`
*   `transformers.AutoModelForSequenceClassification`
*   `transformers.AutoModelForQuestionAnswering`
*   `transformers.AutoModelForTokenClassification`

For text classification we use `transformers.AutoModelForSequenceClassification`:

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=len(labels))
model = model.to(device)
model

As you can see, this model has an **additional linear layer** after the pooler layer. It also comes with built-in loss calculation for convenience:

```Python
loss = None
if labels is not None:
    if self.config.problem\_type is None:
        if self.num\_labels == 1:
            self.config.problem\_type = "regression"
        elif self.num\_labels > 1 and (labels.dtype == torch.long or labels.dtype == torch.int):
            self.config.problem\_type = "single\_label\_classification"
        else:
            self.config.problem\_type = "multi\_label\_classification"

    if self.config.problem\_type == "regression":
        loss\_fct = MSELoss()
        if self.num\_labels == 1:
            loss = loss\_fct(logits.squeeze(), labels.squeeze())
        else:
            loss = loss\_fct(logits, labels)
    elif self.config.problem\_type == "single\_label\_classification":
        loss\_fct = CrossEntropyLoss()
        loss = loss\_fct(logits.view(-1, self.num\_labels), labels.view(-1))
    elif self.config.problem\_type == "multi\_label\_classification":
        loss\_fct = BCEWithLogitsLoss()
        loss = loss\_fct(logits, labels)
```
(*from* [https://github.com/huggingface/transformers/blob/v4.51.3/src/transformers/models/bert/modeling\_bert.py](https://github.com/huggingface/transformers/blob/v4.51.3/src/transformers/models/bert/modeling\_bert.py\#L1692))


Let's try this out:

In [ ]:
model(
  input_ids      = data[:1]['input_ids'].to(device),
  attention_mask = data[:1]['attention_mask'].to(device),
  labels         = data[:1]['label'].to(device)
)

Now let's train this model. Remember, that **in fine-tuning we want to update the pretrained weights, not re-train the model**. Therefore, we train the model for a low number of epochs with a low learning rate. The original BERT paper proposes the following configuration ([Devlin et al, 2019](https://doi.org/10.48550/arXiv.1810.04805)):

* **Learning rate with Adam:** *5e-5, 3e-5, 2e-5*
* **Number of epochs:** *2, 3, 4*

Further reading: *Huggingface also provides a* `transformers.Trainer`*-class that you may want to try out for convenience: [https://huggingface.co/docs/transformers/main\_classes/trainer](https://huggingface.co/docs/transformers/main\_classes/trainer)*

In [ ]:
from torch.optim import AdamW
import tqdm

# Optimizer:
optimizer = AdamW(model.parameters(), lr=5e-5)

# Training loop:
loss_train = []
for epoch in range(2):

  model.train()
  loss_train.append([])
  for batch in tqdm.tqdm(train_dataloader, desc= f" epoch {epoch + 1}"):
    optimizer.zero_grad()

    outputs = model(
      input_ids      = batch['input_ids'].to(device),
      attention_mask = batch['attention_mask'].to(device),
      labels         = batch['label'].to(device)
    )

    loss = outputs.loss
    loss.backward()
    loss_train[-1].append(loss.item())

    optimizer.step()
  loss_train[-1] = np.mean(loss_train[-1])

  print(f"Epoch {epoch+1}: loss = {loss_train[-1]:.3f}")

In [ ]:
# Evaluation:
model.eval()
y_true, y_pred = [], []
for batch in test_dataloader:
  with torch.no_grad():
    outputs = model(
      input_ids      = batch['input_ids'].to(device),
      attention_mask = batch['attention_mask'].to(device)
    )

  y_pred.extend(outputs.logits.argmax(dim=1).cpu().numpy())
  y_true.extend(batch['label'].cpu().numpy())

f1_score(y_true, y_pred, average='macro')

---

<span style="color:red"><b>TASK 2</b> - Fine-Tuning RoBERTa:</span>

---

Fine-tune [RoBERTa](https://huggingface.co/docs/transformers/model\_doc/roberta) a pretrained transformer model to classify the data you received along with this notebook (in `data.zip/task2/...`). **You are enouraged to use scheduling and early stopping, but remember to keep the learning rate low.**

**Upload the resulting predictions to NextIlearn. Your model should achieve an F$\_1$ > .77 to pass.**

In [ ]:
# Load labeled list of training files:
train_files = pd.read_csv('data/task2/train/labels.csv', index_col=0)
train_files['file'] = ['data/task2/train/' + s for s in train_files['file']]
print(f'# of positive samples: {(train_files.label == 1).sum():d}')
print(f'# of negative samples: {(train_files.label == 0).sum():d}')
train_files.head()

In [ ]:
# Load training data sample:
with open(train_files.file.sample(1).iloc[0], 'r') as file:
    print(file.read())

In [ ]:
# Load list of test files:
import os
test_files = ['data/task2/test/' + s for s in os.listdir('data/task2/test/')]
test_files.sort()
test_files = pd.DataFrame({'file': test_files})
test_files.head()

In [ ]:
import os
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from functools import partial
import tqdm


class Task2Dataset(Dataset):
    def __init__(self, label_file, data_file, tokenizer, max_length=128):
        self.labels = pd.read_csv(label_file)
        self.data_file = data_file
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        row = self.labels.iloc[idx]
        file_name = row["file"]
        file_label = row["label"]
        file_text_path = os.path.join(self.data_file, file_name)
        with open(file_text_path, "r", encoding="utf-8") as f:
            file_text = f.read().strip()
        return file_text, file_label


def collate_fn(batch, tokenizer, max_length=256):
    texts, labels = zip(*batch)
    encoded = tokenizer(
        list(texts),
        padding="max_length",
        truncation=True,
        max_length=max_length,
        return_tensors="pt"
    )
    encoded["labels"] = torch.tensor(labels, dtype=torch.long)
    return encoded


def evaluate(model, dataloader, device):
    model.eval()
    y_true, y_pred = [], []
    with torch.no_grad():
        for batch in dataloader:
            batch = {k: v.to(device) for k, v in batch.items()}
            logits = model(**batch).logits
            preds = torch.argmax(logits, dim=1)
            y_pred.extend(preds.cpu().tolist())
            y_true.extend(batch["labels"].cpu().tolist())
    return f1_score(y_true, y_pred, average="macro")


label_file = r"D:\pythonProject\Lab5_empty\data\task2\train\labels.csv"
data_file = r"D:\pythonProject\Lab5_empty\data\task2\train"
tokenizer = AutoTokenizer.from_pretrained("roberta-base")

# Split into train/val CSVs
df = pd.read_csv(label_file)
df_train, df_val = train_test_split(df, test_size=0.2, stratify=df["label"], random_state=42)
df_train.to_csv("train_split.csv", index=False)
df_val.to_csv("val_split.csv", index=False)


train_dataset = Task2Dataset("train_split.csv", data_file, tokenizer)
val_dataset = Task2Dataset("val_split.csv", data_file, tokenizer)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True,
                          collate_fn=partial(collate_fn, tokenizer=tokenizer),
                          )
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False,
                        collate_fn=partial(collate_fn, tokenizer=tokenizer),
                        )


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = AutoModelForSequenceClassification.from_pretrained("roberta-base", num_labels=2)
model.to(device)

optimizer = AdamW(model.parameters(), lr=2e-5, weight_decay=0.01)
num_epochs = 8
best_f1 = 0.0


for epoch in range(num_epochs):
    model.train()
    total_loss, num_batches = 0, 0

    for batch in tqdm.tqdm(train_loader, desc=f"Epoch {epoch+1}"):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        total_loss += loss.item()
        num_batches += 1

    mean_loss = total_loss / num_batches
    print(f"\n Epoch {epoch+1} finished. Mean Loss: {mean_loss:.4f}")

    val_f1 = evaluate(model, val_loader, device)
    print(f" Validation F1-score: {val_f1:.4f}\n")

    if val_f1 > best_f1:
        best_f1 = val_f1
        model.save_pretrained("best_model")
        tokenizer.save_pretrained("best_model")
        print("Saved new best model!")


In [ ]:



device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_task2 = AutoModelForSequenceClassification.from_pretrained("best_model").to(device)
tokenizer = AutoTokenizer.from_pretrained("best_model")

class Task2TestDataset(Dataset):
    def __init__(self, file_dir):
        self.file_names = sorted(os.listdir(file_dir))
        self.file_paths = [os.path.join(file_dir, fname) for fname in self.file_names]
        self.file_ids = [os.path.splitext(fname)[0] for fname in self.file_names]

    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self, idx):
        with open(self.file_paths[idx], "r", encoding="utf-8") as f:
            text = f.read().strip()
        return self.file_ids[idx], text  

def test_collate_fn(batch, tokenizer, max_length=256):
    file_ids, texts = zip(*batch)
    encoded = tokenizer(
        list(texts),
        padding="max_length",
        truncation=True,
        max_length=max_length,
        return_tensors="pt"
    )
    return {
        "file_ids": list(file_ids),
        "input_ids": encoded["input_ids"],
        "attention_mask": encoded["attention_mask"]
    }


test_dataset = Task2TestDataset(r"D:\pythonProject\Lab5_empty\data\task2\test")
test_loader = DataLoader(
    test_dataset,
    batch_size=16,
    shuffle=False,
    collate_fn=partial(test_collate_fn, tokenizer=tokenizer),
    #Every time the num_workers not null will stuck, do not know why
    num_workers=0,  
    pin_memory=False
)


model_task2.eval()
results = []

with torch.no_grad():
    for batch in tqdm.tqdm(test_loader, desc="Predicting"):
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        file_ids = batch["file_ids"]

        logits = model_task2(input_ids=input_ids, attention_mask=attention_mask).logits
        preds = torch.argmax(logits, dim=1).cpu().tolist()

        for fid, pred in zip(file_ids, preds):
            results.append({"file": fid, "predictions": pred})


df = pd.DataFrame(results)
df = df.sort_values("file")  
df = df[["predictions"]]     
df.to_csv("submission.csv", index=True)


---

*End of Task 2. Upload your final predictions (the file* `submission.csv` *) to* **Homework 2 - Code** *on* **NextIlearn**

\#\# 3. Simple Autoregressive Extension of BERT

BERT is not autoregressive by design. Here, we try to simulate next-token prediction using BERT with causal masking.

Remember, that in the MLM pretraining task, BERT is trained to predict the most probable token corresponding to the input token `'[MASK]'`.

E.g.: `'[CLS] The weather is [MASK]. [SEP]'` → **BERT** → `'[CLS] The weather is great. [SEP]'`

We will use this to simulate autoregression, i.e. the generation of a text token by token.

In [ ]:
from transformers import AutoModelForMaskedLM
model = AutoModelForMaskedLM.from_pretrained('bert-base-uncased')
model = model.to(device)
model.eval()
model

An example:

In [ ]:
sentence = "The capital of France is [MASK]."

# tokenize:
inputs = tokenizer(sentence, return_tensors="pt").to(device)

# generate:
with torch.no_grad():
    logits = model(**inputs).logits

logits.shape

The shape of `logits` is:

  *number of texts* $~\times~$ *number of tokens*  $~\times~$  *vocabulary size*

→ This is a collection of token probabilities!

In [ ]:
# get top prediction for masked token:
predicted_token_id = logits[0, -3].argmax(axis=-1)

print("Predicted token:", tokenizer.decode(predicted_token_id))


Now let's make this a function:

In [ ]:
from typing import Optional
def complete_text(prompt:str, max_tokens:Optional[int]=None, model=model, tokenizer=tokenizer, device=device):
  # use the whole context window if max_tokens not specified:
  if max_tokens is None: max_tokens = tokenizer.model_max_length - len(tokenizer(prompt).input_ids)

  # pad prompt with '[MASK]' tokens to tell BERT the number of tokens:
  prompt += ' '.join(['[MASK]']*max_tokens)

  # tokenize:
  inputs = tokenizer(prompt, return_tensors="pt").to(device)

  # generate token probabilities:
  with torch.no_grad():
    logits = model(**inputs).logits

  # get top prediction for first masked token:
  predicted_token_id = logits[0, -max_tokens-1].argmax(axis=-1).cpu().tolist()

  text = tokenizer.decode(inputs.input_ids[0, 1:-max_tokens-1].cpu().tolist() + [predicted_token_id])

  # end autoregression if max_tokens == 1:
  if max_tokens == 1: return text

  # end autoregression on '.' token:
  if predicted_token_id == tokenizer.vocab['.']: return text

  # end autoregression on [SEP] token:
  if predicted_token_id == tokenizer.sep_token_id: return text

  return complete_text(text, max_tokens=max_tokens-1, model=model, tokenizer=tokenizer, device=device)

Let's see if it works:

In [ ]:
complete_text("The capital of France is ", max_tokens=2)

In [ ]:
complete_text("The capital of France is ", max_tokens=100)

Another example:

In [ ]:
complete_text("one plus one is equal to ", max_tokens=2)

In [ ]:
complete_text("one plus one is equal to ")